In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl

# making class similarity

In [ ]:
class Similarity:

    def __init__(self,
                 arrays,
                 choice='cosine'):

        self.vector = arrays
        self.choice = choice
        assert self.vector.shape == (5,)

    # using cosine similarity algorithm
    def cosine(self, vectors):

        result = np.array([])

        for vector in vectors:
            dot_product = np.dot(self.vector, vector)
            norm_vector = np.linalg.norm(self.vector)
            norm_vector2 = np.linalg.norm(vector)

            cosine_similarity = dot_product / (norm_vector * norm_vector2)
            result = np.append(result, [cosine_similarity])

        return result

    # using euclidean distance algorithm
    def euclidean(self, vectors):

        result = np.array([])

        for vector in vectors:
            euclidean_distance = np.linalg.norm(self.vector - vector)
            result = np.append(result, [euclidean_distance])

        return result

    # start to find similarity
    def find_similiarity(self, vectors):

        if self.choice == "cosine":
            result = self.cosine(vectors)

        if self.choice == "euclidean":
            result = self.euclidean(vectors)

        return result

# making class recommendation

In [ ]:
class Recommendation:

    def __init__(self,
                 df,
                 cols,
                 vector,
                 choice='cosine'):

        self.vector = vector
        self.cols = cols
        self.data = df[self.cols].values
        self.choice = choice

        assert len(cols) == len(vector)

    # find recommendation by calling class similarity
    def find_recommendation(self, amount):

        self.vector = np.array(self.vector)
        similarity = Similarity(self.vector, choice=self.choice)
        df = pd.DataFrame(self.data,
                          columns=self.cols)
        df['similarity'] = similarity.find_similiarity(self.data)

        if self.choice == 'cosine':
            df = df.sort_values('similarity', ascending=False).drop(columns=['similarity']).tail(-1)
            return df.head(amount)

        if self.choice == 'euclidean':
            df = df.sort_values('similarity', ascending=True).drop(columns=['similarity']).tail(-1)
            return df.head(amount)

In [ ]:
import mysql.connector as sql

In [ ]:
def fetch_db(host, user, password, database):

    mydb = sql.connect(
        host=host,
        user=user,
        password=password,
        database=database
      )

    cursor = mydb.cursor()

    query = '''SELECT `talentName`, `email`, `latitude`, `longitude`,
              `price`, `categoryId`, `quantity` from tb_talents'''

    query_long = '''SELECT max(`talentId`) from tb_talents'''

    cursor.execute(query_long)
    length = cursor.fetchmany()[0][0]

    cursor.execute(query)
    result = cursor.fetchmany(size=length)

    mydb.close()
    return result

In [ ]:
# find the recommendation by top 10 most similar result (json formatted)
def json_result(fetch_result, vector):

    quantity = {"single":0,
            "duo":1,
            "group":2}

    df = {'name':np.array([]),
        'email':np.array([]),
        'latitude':np.array([]),
        'longitude':np.array([]),
        'price':np.array([]),
        'category':np.array([]),
        'quantity':np.array([])}

    for i in fetch_result:

        (name, email, latitude, longitude, price, category, qty) = i
        df['name'] = np.append(df['name'], name)
        df['email'] = np.append(df['email'], email)
        df['latitude'] = np.append(df['latitude'], latitude)
        df['longitude'] = np.append(df['longitude'], longitude)
        df['price'] = np.append(df['price'], price)
        df['category'] = np.append(df['category'], category)
        df['quantity'] = np.append(df['quantity'], quantity[qty])

    df = pd.DataFrame(df)
    df_test = df[df.columns[2:]]
    recc = Recommendation(df_test,
                        df_test.columns,
                        vector,
                        choice='cosine')

    result = recc.find_recommendation(10).reset_index()
    result = df[df.columns[:2]].reset_index().merge(result, on='index').drop(columns=['index'])
    result = result.to_json()

    return result

In [ ]:
import os
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request

# making flask app
app = Flask(name)
app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESPONSE'] = 'response.json'

host="localhost"
user="root"
password=""
database="db_talent"

result = fetch_db(host, user, password, database)

def json_data(db_result, vector):

    json = json_result(db_result, vector)
    return json


@app.route("/")
def index():
    return "Hello World!"


@app.route("/prediction", methods=["POST"])
def prediction_route():
    if request.method == "POST":

        json = request.json
        if json is not None:
            vector = np.array([json['latutude'],
                           json['longitude'],
                           json['price'],
                           json['category'],
                           json['quantity'],])
            json_vector = json_data(result, vector)

            return jsonify({
                "status": {
                    "code": 200,
                    "message": "Success predicting"
                },
                "data": {
                    "recommendation": json_vector,
                }
            }), 200
        else:
            return jsonify({
                "status": {
                    "code": 400,
                    "message": "Text should not be none."
                },
                "data": None,
            }), 400
    else:
        return jsonify({
            "status": {
                "code": 405,
                "message": "Method not allowed"
            },
            "data": None,
        }), 405


if __name__ == "main":
    app.run(debug=True,
            host="0.0.0.0",
            port=int(os.environ.get("PORT", 8080)))